# Evaluation test set playground

In [2]:
import scipy.io
import math
import os
import glob
import numpy as np
import json
import pandas as pd
from birdnetlib.analyzer import Analyzer
from birdnetlib.batch import DirectoryAnalyzer

In [21]:
# E:\Giacomo\Tovanella-20241110T120546Z-001\training_sessions\session_0\test\Certhia familiaris_Eurasian Treecreeper\20200217_160000_591.wav
# salva tutti i risultati su questo array
pred_segments = {}

def on_analyze_complete(recording):
    # E:\Giacomo\Tovanella-20241110T120546Z-001\test_set\test\Periparus ater_Coal Tit\20200215_090000_259.wav
    audio_name = recording.path.split('\\')[-1]
    date, number, start_time = audio_name.split('_')    # <20200215>_<090000>_<259.wav>
    start_time = start_time.split('.')[0]               # <259>.<wav>
    segm = int(start_time) // 3
    audio_name = "_".join([date, number]) + ".WAV"      # 20200215_090000.WAV
    if audio_name not in pred_segments:
        pred_segments[audio_name] = {}
    if segm not in pred_segments[audio_name]:
        pred_segments[audio_name][segm] = []
    pred_segments[audio_name][segm].extend([detection["label"] for detection in recording.detections])
    print("Analyzing ", recording.path)

def on_error(recording, error):
    print("An exception occurred: {}".format(error))
    print(recording.path)

In [22]:
pred_segments = {}
model_path = "classifiers\\new_segments\\CustomClassifier.tflite"
labels_path = "classifiers\\new_segments\\CustomClassifier_Labels.txt"
audio_path = "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\test_set_augm\\test"

analyzer = Analyzer(
    classifier_labels_path=labels_path, 
    classifier_model_path=model_path,
)    

for folder in os.listdir(audio_path):
    directory = os.path.join(audio_path, folder)

    print("Starting Watcher: ", folder)
    batch = DirectoryAnalyzer(
        directory,
        analyzers=[analyzer],
        min_conf=0.1,
    )

    batch.on_analyze_complete = on_analyze_complete
    batch.on_error = on_error
    batch.process()

load_custom_models
Custom model loaded.
loading custom classifier labels
Labels loaded.
load model False
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.
Starting Watcher:  Certhia familiaris_Eurasian Treecreeper
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190621_170000_10.wav
Analyzing  E:\Giacomo\Tovanella-20241110T120546Z-001\test_set_augm\test\Certhia familiaris_Eurasian Treecreeper\20190621_170000_10.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190621_170000_15.wav
Analyzing  E:\Giacomo\Tovanella-20241110T120546Z-001\test_set_augm\test\Certhia familiaris_Eurasian Treecreeper\20190621_170000_15.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording 20190621_170000_17.wav
Analyzing  E:\Giacomo\Tovanella-20241110T120546Z-001\test_set_augm\test\Certhia familiaris_Eurasian Treecreeper\20190621_170000_17.wav
read_audio_data
read_audio_data: complete, read  1 chunks.


In [23]:
pred_segments

{'20190621_170000.WAV': {3: ['Regulus ignicapilla_Common Firecrest',
   'Sylvia atricapilla_Eurasian Blackcap',
   'Fringilla coelebs_Common Chaffinch'],
  5: ['Fringilla coelebs_Common Chaffinch',
   'Phylloscopus collybita_Common Chiffchaff',
   'Sylvia atricapilla_Eurasian Blackcap',
   'Fringilla coelebs_Common Chaffinch'],
  6: ['Sylvia atricapilla_Eurasian Blackcap',
   'Fringilla coelebs_Common Chaffinch',
   'Phylloscopus collybita_Common Chiffchaff',
   'Fringilla coelebs_Common Chaffinch',
   'Sylvia atricapilla_Eurasian Blackcap',
   'Regulus ignicapilla_Common Firecrest',
   'Fringilla coelebs_Common Chaffinch'],
  7: ['Sylvia atricapilla_Eurasian Blackcap',
   'Fringilla coelebs_Common Chaffinch',
   'Fringilla coelebs_Common Chaffinch',
   'Sylvia atricapilla_Eurasian Blackcap',
   'Fringilla coelebs_Common Chaffinch',
   'Wind_'],
  8: ['Fringilla coelebs_Common Chaffinch',
   'Loxia curvirostra_Common Crossbill',
   'Sylvia atricapilla_Eurasian Blackcap',
   'Fringilla 

In [24]:
from sklearn.preprocessing import MultiLabelBinarizer

species_list = os.listdir(audio_path)

mlb = MultiLabelBinarizer()
mlb.fit([list(species_list)])

len(mlb.classes_)

19

In [42]:
with open("utils/segments_info.json") as f:
    true_segments = json.load(f)

In [43]:
y_pred = []
y_true = []
y_pred_audios = {} 
for audio in pred_segments:
    y_pred.append(mlb.transform(pred_segments[audio].values()))     # apply transform on every label of every segment
    y_true_audio = [labels for segm, labels in true_segments[audio].items() if int(segm) in pred_segments[audio]]
    y_true.append(mlb.transform(y_true_audio))

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

In [44]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)
# apply colors
# add index column name to df
report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Aeroplane_,0.000000,0.000000,0.000000,18.0
Certhia familiaris_Eurasian Treecreeper,0.035294,0.375000,0.064516,8.0
Dendrocopos major_Great Spotted Woodpecker,0.047619,0.166667,0.074074,6.0
Dryocopus martius_Black Woodpecker,0.011236,0.333333,0.021739,3.0
Erithacus rubecula_European Robin,0.009091,0.142857,0.017094,7.0
Fringilla coelebs_Common Chaffinch,0.175824,0.190476,0.182857,84.0
Glaucidium passerinum_Eurasian Pygmy-Owl,0.000000,0.000000,0.000000,10.0
Insect_,0.000000,0.000000,0.000000,2.0
Lophophanes cristatus_Crested Tit,0.200000,0.500000,0.285714,4.0
Loxia curvirostra_Common Crossbill,0.000000,0.000000,0.000000,16.0


In [45]:
print(classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0))

                                            precision    recall  f1-score   support

                                Aeroplane_       0.00      0.00      0.00        18
   Certhia familiaris_Eurasian Treecreeper       0.04      0.38      0.06         8
Dendrocopos major_Great Spotted Woodpecker       0.05      0.17      0.07         6
        Dryocopus martius_Black Woodpecker       0.01      0.33      0.02         3
         Erithacus rubecula_European Robin       0.01      0.14      0.02         7
        Fringilla coelebs_Common Chaffinch       0.18      0.19      0.18        84
  Glaucidium passerinum_Eurasian Pygmy-Owl       0.00      0.00      0.00        10
                                   Insect_       0.00      0.00      0.00         2
         Lophophanes cristatus_Crested Tit       0.20      0.50      0.29         4
        Loxia curvirostra_Common Crossbill       0.00      0.00      0.00        16
      Muscicapa striata_Spotted Flycatcher       0.85      0.78      0.81  